# -*- coding: utf-8 -*-
"""
Conexión a PostgreSQL, Creación y Poblamiento de Tabla, y Carga con Pandas y Polars en Google Colab (Local)

Este cuaderno Jupyter (Google Colab) demuestra cómo conectarse a una base de datos PostgreSQL local,
crear una tabla, insertar 100 registros de ejemplo, y luego cargar esos datos
usando las librerías Pandas y Polars. El código está diseñado para ser
ejecutado localmente en un PC.

Instrucciones Previas:
1. Asegúrate de tener PostgreSQL instalado y ejecutándose en tu máquina local.
2. Crea una base de datos (ej. 'red') y un usuario (ej. 'mi_usuario')
   con una contraseña (ej. '123456') con los permisos adecuados.
   Puedes hacerlo desde la línea de comandos de PostgreSQL (psql):
   ```sql
   CREATE DATABASE red;
   CREATE USER mi_usuario WITH PASSWORD '123456';
   GRANT ALL PRIVILEGES ON DATABASE red TO mi_usuario;
   ```
3. Instala las librerías necesarias en tu entorno local si no las tienes:
   `pip install psycopg2-binary pandas polars Faker`
"""

In [58]:
%pip install psycopg2-binary pandas polars Faker
%pip install SQLAlchemy

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\dmazo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: C:\Users\dmazo\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [59]:
# 1. Importar Librerías Necesarias
# ---------------------------------
import psycopg2
import pandas as pd
import polars as pl
from faker import Faker
import random
from datetime import datetime
from sqlalchemy import create_engine

In [60]:
# 2. Configuración de la Conexión a la Base de Datos PostgreSQL (Local)
# -------------------------------------------------------------------
# Asegúrate de que estos parámetros coincidan con tu configuración local de PostgreSQL.
DB_HOST     = "localhost"   # La dirección IP de tu servidor PostgreSQL (normalmente 'localhost' para local)
DB_NAME     = "db12"         # El nombre de la base de datos a la que te quieres conectar
DB_USER     = "postgres"  # El nombre de usuario de la base de datos
DB_PASSWORD = "postgres"      # La contraseña del usuario de la base de datos
DB_PORT     = "5432"        # El puerto de tu servidor PostgreSQL (el predeterminado es 5432)

In [61]:
# --------------------------------------------------------
# 3. Función para Establecer la Conexión a la Base de Datos
# Establece y devuelve una conexión a la base de datos PostgreSQL.
# --------------------------------------------------------
def get_db_connection():
    try:
        print(f"DEBUG - DB_HOST: '{DB_HOST}'")
        print(f"DEBUG - DB_NAME: '{DB_NAME}'")
        print(f"DEBUG - DB_USER: '{DB_USER}'")
        # ¡IMPORTANTE! No imprimas la contraseña directamente por seguridad.
        # Solo verifica que tiene un valor y que no está vacía.
        print(f"DEBUG - DB_PASSWORD length: {len(DB_PASSWORD) if DB_PASSWORD else 0}")
        print(f"DEBUG - DB_PASSWORD starts with: '{DB_PASSWORD[0]}' ends with: '{DB_PASSWORD[-1]}'")
        print(f"DEBUG - DB_PORT: '{DB_PORT}'")

        conn = psycopg2.connect(
            host    =DB_HOST,
            database=DB_NAME,
            user    =DB_USER,
            password=DB_PASSWORD,
            port    =DB_PORT
        )
        print("Conexión a la base de datos PostgreSQL exitosa!")
        return conn
    except psycopg2.Error as e:
        print(f"Error al conectar a la base de datos PostgreSQL: {e}")
        return None

In [62]:
# ---------------------------------------------
# 4. Creación de la Tabla 'estudiantes'
#    Crea la tabla 'estudiantes' si no existe.
#    Se pasa el parámetro de conexión a la BD
# ---------------------------------------------
def crear_tabla_usuario(conn):
    cursor = conn.cursor()
    try:
        cursor.execute("""
            CREATE TABLE usuario (
                cod_usuario INT PRIMARY KEY,
                usuario VARCHAR(50) NOT NULL UNIQUE,
                contrasena_usuario VARCHAR(20) NOT NULL
            );
        """)
        conn.commit()
        print("Tabla 'usuario' creada o ya existe exitosamente.")
    except psycopg2.Error as e:
        print(f"Error al crear la tabla 'usuario': {e}")
        conn.rollback() # Deshacer la transacción en caso de error
    finally:
        cursor.close()

In [63]:
#from faker import Faker
import psycopg2

def cargar_muestra_usuarios(conn, num_records=100):
    fake = Faker('es_CO')

    print(f"\nInsertando {num_records} registros en la tabla 'usuario'...")

    try:
        with conn.cursor() as cursor:
            # Obtener el último cod_usuario existente
            cursor.execute("SELECT MAX(cod_usuario) FROM usuario;")
            last_cod_usuario = cursor.fetchone()[0]
            next_cod_usuario = (last_cod_usuario if last_cod_usuario is not None else 0) + 1

            for i in range(num_records):
                v_cod_usuario = next_cod_usuario + i
                v_usuario = fake.user_name()
                v_contrasena_usuario = fake.password(length=10, special_chars=True, digits=True, upper_case=True, lower_case=True)

                try:
                    cursor.execute("""
                        INSERT INTO usuario (cod_usuario, usuario, contrasena_usuario)
                        VALUES (%s, %s, %s)
                        ON CONFLICT (usuario) DO NOTHING;
                    """, (v_cod_usuario, v_usuario, v_contrasena_usuario))
                except psycopg2.Error as e:
                    print(f"Error al insertar cod_usuario={v_cod_usuario}: {e}")
                    conn.rollback()

            conn.commit()
            print(f"{num_records} registros de usuarios intentados. Revise las advertencias para los fallos.")

    except Exception as e:
        print(f"Error crítico en la inserción de usuarios: {e}")
        conn.rollback()

In [64]:
# ----------------------------------------
# 6. Despliegue y Carga de Datos con Pandas
#    Carga los datos de la tabla 'estudiantes' en un DataFrame de Pandas.
# ----------------------------------------
def cargar_datos_con_pandas(conn):
    print("\nCargando datos con Pandas...")
    try:
        df_pandas = pd.read_sql_query("SELECT * FROM usuario;", conn)
        print("Datos cargados exitosamente en un DataFrame de Pandas.")
        print("Primeras 10 filas del DataFrame de Pandas:")
        print(df_pandas.head(10))
        print(f"Número de filas en el DataFrame de Pandas: {len(df_pandas)}")
        return df_pandas
    except Exception as e:
        print(f"Error al cargar datos con Pandas: {e}")
        return None

In [65]:
# -------------------------------------
# 7. Función Principal para la Ejecución
#    Función principal que orquesta la conexión, creación, poblamiento y carga de datos.
# -------------------------------------
def main():
    conn = None # Inicializar la conexión a None
    try:
        conn = get_db_connection()
        if conn:
            crear_tabla_usuario(conn)
            cargar_muestra_usuarios(conn, num_records=100)

            # Cargar y mostrar datos con Pandas
            df_pandas = cargar_datos_con_pandas(conn)
    finally:
        if conn:
            conn.close()
            print("\nConexión a la base de datos cerrada.")

In [66]:
# -------------------------------
# 8. Ejecutar la Función Principal
# -------------------------------
if __name__ == "__main__":
    main()


DEBUG - DB_HOST: 'localhost'
DEBUG - DB_NAME: 'db12'
DEBUG - DB_USER: 'postgres'
DEBUG - DB_PASSWORD length: 8
DEBUG - DB_PASSWORD starts with: 'p' ends with: 's'
DEBUG - DB_PORT: '5432'
Conexión a la base de datos PostgreSQL exitosa!
Error al crear la tabla 'usuario': la relación «usuario» ya existe


Insertando 100 registros en la tabla 'usuario'...
100 registros de usuarios intentados. Revise las advertencias para los fallos.

Cargando datos con Pandas...
Datos cargados exitosamente en un DataFrame de Pandas.
Primeras 10 filas del DataFrame de Pandas:
   cod_usuario      usuario contrasena_usuario
0            1     eduard13         @1VBL7rMvU
1            2   olgasuarez         #76Q)#Uhe3
2            3    mariana87         T!^8+Napq$
3            4       yortiz         $k8sNwVc%6
4            5  alexander32         *8MLABfGM4
5            6     rcabrera         4gEaflT0^b
6            7   davidortiz         +1kD7k_oY1
7            8  castrodiego         6R&!74Dcf$
8            9  

C:\Users\dmazo\AppData\Local\Temp\ipykernel_32464\3628179052.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_pandas = pd.read_sql_query("SELECT * FROM usuario;", conn)
